In [2]:
# Importamos las librerias a usar
import pandas as pd
import json
import ast
import re
import nltk


In [3]:
# Creo un lista donde se depositaran los datos
reviewsList = []
# Abro un archivo, señalizando que tiene un encoding 'utf-8'
with open('australian_user_reviews.json','r', encoding='utf-8') as f:
    # Leo cada linea con el bucle for
  for line in f.readlines():
      # lee todas las líneas del archivo y las coloca en una lista de cadenas.
    reviewsList.append(ast.literal_eval(line))
# Convierto el archivo a un data frame
dfReviews = pd.DataFrame(reviewsList)
# Abro los archivos
dfReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
#Informacion general del dataframe
filas,columnas = dfReviews.shape
print("El numero de filas : ",filas)
print("El numero de columnas : ",columnas,'\n')
print("Informacion general de la tabla dfReviews\n")
print(dfReviews.info())

El numero de filas :  25799
El numero de columnas :  3 

Informacion general de la tabla dfReviews

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB
None


In [5]:
#Compruebo cantidad de nulos
dfReviews.isna().sum() # No tiene nulos

user_id     0
user_url    0
reviews     0
dtype: int64

In [6]:
##Eliminar las columnas menos relevantes del Dataframe Items
columnasAEliminar = ['user_url']
dfReviews = dfReviews.drop(columns=columnasAEliminar)
dfReviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [7]:
# Explorar lo datos de reviews
dfReviews = dfReviews.explode('reviews')
dfReviews
#Desanidamos la columna reviews 
dfReviews = pd.json_normalize(json.loads(dfReviews.to_json(orient = "records")))
# Eliminar columna Reviews
dfReviews = dfReviews.drop(columns=['reviews'])
# Eliminar filas nulas
dfReviews = dfReviews.dropna()
dfReviews

,user_id,reviews.funny,reviews.posted,reviews.last_edited,reviews.item_id,reviews.helpful,reviews.recommend,reviews.review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [8]:
# Columna 'reviews.funny'
#Contar tipos de palabra
conteo = dfReviews['reviews.funny'].value_counts()
conteo

# Eliminar las siguientes cadena de texto y espacios vacios
eliminarFunny1='person found this review funny'
eliminarFunny2 = 'people found this review funny'
eliminarFunny3 = ','
dfReviews['reviews.funny'] = dfReviews['reviews.funny'].str.replace(eliminarFunny1, '')
dfReviews['reviews.funny'] = dfReviews['reviews.funny'].str.replace(eliminarFunny2, '')
dfReviews['reviews.funny'] = dfReviews['reviews.funny'].str.replace(eliminarFunny3, '')
dfReviews['reviews.funny'] = dfReviews['reviews.funny'].str.strip()

# Reemplazar valores vacios y con cero y convertir a tipo entero
dfReviews['reviews.funny'] = dfReviews['reviews.funny'].replace('', '0')
#Convertir a Entero
dfReviews['reviews.funny'].astype(int)

0        0
1        0
2        0
3        0
4        0
        ..
59328    0
59329    0
59330    1
59331    0
59332    0
Name: reviews.funny, Length: 59305, dtype: int32

In [9]:
# Columna  'reviews.posted'

# Eliminar las siguientes cadena de texto y espacios vacios
eliminarPosted='Posted '
eliminarPosted2 = '.'
dfReviews['reviews.posted'] = dfReviews['reviews.posted'].str.replace(eliminarPosted, '')
dfReviews['reviews.posted'] = dfReviews['reviews.posted'].str.replace(eliminarPosted2, '')

# Fecha para los sin fecha (el punto medio entre la fecha minima y maxima de los posted qeu si tiene fecha)
enteros_extraidos = []
for fila in dfReviews['reviews.posted']:  
  ultimos_cuatro = fila[-4:]

  try:
    entero = int(ultimos_cuatro)
    enteros_extraidos.append(entero)
  except ValueError:
    pass
maximo_enteros = max(enteros_extraidos, default=None)
minimo_enteros = min(enteros_extraidos, default=None)
promedio = str(int((maximo_enteros+minimo_enteros)/2))      

# Reemplazar las filas con la fecha indicado, y si no ponerlas al por defecto al año 1900
def condicion_fila(fila):
    if ',' in fila:
        return pd.to_datetime(fila,format='%B %d, %Y')
    else:        
        try:
          return pd.to_datetime(fila + ', '+promedio,format='%B %d, %Y')
        except ValueError:
          return None  # Devolver None para fechas inválidas

# Aplicar la función a cada fila utilizando apply()
dfReviews['reviews.posted'] = dfReviews['reviews.posted'].apply(condicion_fila)

In [10]:
# Columna  'reviews.last_edited'
#Contar tipos de palabra
conteo = dfReviews['reviews.last_edited'].value_counts()
conteo

# Eliminar las siguientes cadena de texto y espacios vacios
eliminaredited='Last edited '
eliminaredited2 = '.'
dfReviews['reviews.last_edited'] = dfReviews['reviews.last_edited'].str.replace(eliminaredited, '')
dfReviews['reviews.last_edited'] = dfReviews['reviews.last_edited'].str.replace(eliminaredited2, '')

# Fecha para los sin fecha
enteros_extraidos = []
for fila in dfReviews['reviews.last_edited']:  
  ultimos_cuatro = fila[-4:]

  try:
    entero = int(ultimos_cuatro)
    enteros_extraidos.append(entero)
  except ValueError:
    pass
maximo_enteros = max(enteros_extraidos, default=None)
minimo_enteros = min(enteros_extraidos, default=None)
promedio = str(int((maximo_enteros+minimo_enteros)/2))      

# Reemplazar las filas con la fecha indicado, y si no ponerlas al por defecto al año 1900
def condicion_fila(fila):
    if ',' in fila:
        return pd.to_datetime(fila,format='%B %d, %Y')
    else:        
        try:
          return pd.to_datetime(fila + ', '+promedio,format='%B %d, %Y')
        except ValueError:
          return None  # Devolver None para fechas inválidas

# Aplicar la función a cada fila utilizando apply()
dfReviews['reviews.last_edited'] = dfReviews['reviews.last_edited'].apply(condicion_fila)

In [11]:
#Extraer la calificaion de hepful
import re
def extraer_texto_entre_parentesis(texto):
    match = re.search(r'\((.*?)\)', texto)
    if match:
        return match.group(1)
    else:
        return ""
# Aplicar la función a la columna "texto" y crear una nueva columna "texto_extraido"
dfReviews['reviews.helpful'] = 	dfReviews['reviews.helpful'].apply(extraer_texto_entre_parentesis)
def porcentaje_a_decimal(valor_str):
    # Verificar si la cadena está vacía
    if valor_str == '':
        return None  # Otra opción es devolver un valor especial o un valor por defecto
    # Remover el símbolo de porcentaje y luego convertir a decimal
    valor_str = valor_str.strip('%')
    return float(valor_str) / 100  # Dividir por 100 para obtener el valor decimal
# Aplicar la función a la columna 'porcentaje' y crear una nueva columna 'decimal'
dfReviews['reviews.helpful'] = dfReviews['reviews.helpful'].apply(porcentaje_a_decimal)

In [12]:
#Convertir reviews.recommend 0 y 1
dfReviews['reviews.recommend'] = dfReviews['reviews.recommend'].astype(int)

In [13]:
#Analisis de sentimiento en ingles
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
#dfReviews['reviews.review'] = dfReviews['reviews.review'].apply(lambda i: sid.polarity_scores(i)['compound'])
dfReviews['sentimiento'] = dfReviews['reviews.review'].apply(lambda x: sid.polarity_scores(x)['compound'])
# Realizar el análisis de sentimiento
# Comnvertir el valor de los sentimientos a 0(malo),1(neutral),2(positivo)
def convertirSentimiento(valor):
    if valor < 0:
        return 0
    elif valor > 0:
        return 2
    else:
        return 1
#Convertir tipo entero
dfReviews['sentimiento'] = dfReviews['sentimiento'].apply(convertirSentimiento).astype(int)

#Eliminar comentarios 
eliminarComentarios = ['reviews.review']
dfReviews = dfReviews.drop(columns=eliminarComentarios)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
dfReviews 

,user_id,reviews.funny,reviews.posted,reviews.last_edited,reviews.item_id,reviews.helpful,reviews.recommend,sentimiento
0,76561197970982479,0,2011-11-05,NaT,1250,NaN,1,2
1,76561197970982479,0,2011-07-15,NaT,22200,NaN,1,2
2,76561197970982479,0,2011-04-21,NaT,43110,NaN,1,2
3,js41637,0,2014-06-24,NaT,251610,0.75,1,2
4,js41637,0,2013-09-08,NaT,227300,0.00,1,2
...,...,...,...,...,...,...,...,...
59328,76561198312638244,0,2012-07-10,NaT,70,NaN,1,2
59329,76561198312638244,0,2012-07-08,NaT,362890,NaN,1,2
59330,LydiaMorley,1,2012-07-03,NaT,273110,0.50,1,2
59331,LydiaMorley,0,2012-07-20,NaT,730,NaN,1,2


In [15]:
dfReviews.to_csv('dfReviewsLimpio.csv', index=False)